In [1]:
import shutil
import os

# Define the working directory
working_dir = '/working'

# Delete all contents of the working directory
for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)  # Remove file or link
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # Remove directory
    except Exception as e:
        print(f"Failed to delete {item_path}. Reason: {e}")

print("Cleaned all files and directories in '/working'.")

Cleaned all files and directories in '/working'.


## Importando Bibliotecas

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

## Definir caminhos dos conjuntos de dados

In [3]:
original_dataset_path = 'PlantVillage'
balanced_dataset_path = '/working/PlantVillage'
train_dir = '/working/train'
test_dir = '/working/test'

# Essa função copia todo o diretório especificado em original_dataset_path
# (incluindo subdiretórios e arquivos) para o caminho balanced_dataset_path
shutil.copytree(original_dataset_path, balanced_dataset_path)

'/working/PlantVillage'

In [4]:
print(os.path.exists(original_dataset_path))  # Deve retornar True
print(os.path.exists(balanced_dataset_path))  # Verificar se já existe


True
True


## Definir geradores de dados

In [5]:
# Criação de um gerador de dados para o conjunto de treinamento
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Redimensiona os valores dos pixels para o intervalo [0, 1], já que os pixels originais estão no intervalo [0, 255].
    rotation_range=40,          # Rotação aleatória das imagens em até 40 graus.
    width_shift_range=0.2,      # Translada a imagem horizontalmente em até 20% da largura total.
    height_shift_range=0.2,     # Translada a imagem verticalmente em até 20% da altura total.
    shear_range=0.2,            # Aplica cisalhamento (shear) na imagem em até 20%.
    zoom_range=0.2,             # Aplica zoom aleatório na imagem em até 20%.
    horizontal_flip=True,       # Permite a inversão horizontal das imagens (flip).
    fill_mode='nearest'         # Define como preencher os pixels vazios gerados por transformações; 'nearest' usa os pixels mais próximos.
)

# Criação de um gerador de dados para o conjunto de teste
test_datagen = ImageDataGenerator(
    rescale=1./255              # Redimensiona os valores dos pixels para o intervalo [0, 1].
    # Observação: aqui não há outras transformações, pois o conjunto de teste deve ser usado "como está" para avaliar o desempenho real do modelo.
)



## Divida os dados em treinamento e teste (70/30)

In [6]:
# Lista todas as categorias (subdiretórios) dentro do dataset balanceado
categories = [d for d in os.listdir(balanced_dataset_path) if os.path.isdir(os.path.join(balanced_dataset_path, d))]

# Cria os diretórios para armazenar os conjuntos de treinamento e teste
os.makedirs(train_dir, exist_ok=True)  # Cria o diretório de treinamento, se ainda não existir
os.makedirs(test_dir, exist_ok=True)  # Cria o diretório de teste, se ainda não existir

# Para cada categoria encontrada
for category in categories:
    # Define os caminhos para a categoria atual no dataset balanceado e nos conjuntos de treinamento/teste
    category_path = os.path.join(balanced_dataset_path, category)  # Caminho da categoria no dataset original
    category_train_path = os.path.join(train_dir, category)        # Caminho da categoria no conjunto de treinamento
    category_test_path = os.path.join(test_dir, category)          # Caminho da categoria no conjunto de teste

    # Cria os subdiretórios para a categoria no conjunto de treinamento e teste
    os.makedirs(category_train_path, exist_ok=True)  # Cria o subdiretório de treinamento para a categoria, se não existir
    os.makedirs(category_test_path, exist_ok=True)   # Cria o subdiretório de teste para a categoria, se não existir

    # Lista todos os arquivos na categoria atual
    files = os.listdir(category_path)

    # Define o ponto de divisão: 70% dos arquivos para treinamento, 30% para teste
    split_point = int(0.7 * len(files))  # Calcula a posição onde será feita a divisão

    # Move os primeiros 70% dos arquivos para o conjunto de treinamento
    for file in files[:split_point]:
        shutil.move(os.path.join(category_path, file), os.path.join(category_train_path, file))  # Move o arquivo

    # Move os 30% restantes dos arquivos para o conjunto de teste
    for file in files[split_point:]:
        shutil.move(os.path.join(category_path, file), os.path.join(category_test_path, file))  # Move o arquivo


## Carregar dados de treinamento e teste

In [7]:
# Criação de um gerador para o conjunto de treinamento
train_generator = train_datagen.flow_from_directory(
    train_dir,                # Caminho do diretório que contém o conjunto de treinamento
    target_size=(224, 224),   # Redimensiona as imagens para o tamanho 224x224 pixels
    batch_size=32,            # Número de imagens processadas em cada iteração (batch)
    class_mode='categorical', # Define que as classes são categóricas (uma classe por imagem, codificada como one-hot)
    shuffle=True              # Embaralha as imagens para melhorar a generalização do modelo
)

# Criação de um gerador para o conjunto de teste
test_generator = test_datagen.flow_from_directory(
    test_dir,                 # Caminho do diretório que contém o conjunto de teste
    target_size=(224, 224),   # Redimensiona as imagens para o tamanho 224x224 pixels
    batch_size=32,            # Número de imagens processadas em cada iteração (batch)
    class_mode='categorical', # Define que as classes são categóricas (uma classe por imagem, codificada como one-hot)
    shuffle=False             # Não embaralha as imagens; necessário para manter a ordem ao avaliar o modelo
)

Found 26647 images belonging to 16 classes.
Found 14629 images belonging to 16 classes.


## Implementar SMOTE (opcional, se necessário)

In [10]:
# Importa o modelo VGG16 e cria um extrator de características
feature_extractor = Sequential([
    VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Usa a VGG16 pré-treinada no ImageNet
])
feature_extractor.trainable = False  # Congela os pesos do modelo para não serem atualizados durante o treinamento

# Inicializa listas para armazenar as características extraídas e os rótulos
features, labels = [], []

# Itera pelos lotes (batches) do gerador de treinamento
for batch, label_batch in train_generator:
    # Usa o modelo VGG16 para extrair características do lote de imagens
    feature_batch = feature_extractor.predict(batch)
    
    # Achata (flatten) as características extraídas para que fiquem em um formato bidimensional
    features.append(feature_batch.reshape((feature_batch.shape[0], -1)))  # Cada imagem vira um vetor
    labels.append(label_batch)  # Adiciona os rótulos correspondentes

    # Verifica se todos os lotes foram processados (o número de lotes é igual ao comprimento do gerador)
    if len(features) >= len(train_generator): 
        break

# Concatena todas as características e rótulos em arrays únicos
features = np.vstack(features)  # Une todos os lotes de características
labels = np.argmax(np.vstack(labels), axis=1)  # Converte rótulos one-hot para índices de classes

# Usa a técnica SMOTE para balancear as classes nos dados de treinamento
smote = SMOTE()  # Inicializa o algoritmo SMOTE
X_resampled, y_resampled = smote.fit_resample(features, labels)  # Gera novos exemplos para equilibrar as classes

# Divide os dados balanceados em conjuntos de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(
    X_resampled,      # Características balanceadas
    y_resampled,      # Rótulos balanceados
    test_size=0.3,    # 30% dos dados são reservados para validação
    random_state=42   # Define uma semente aleatória para reprodução
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 

MemoryError: Unable to allocate 1.08 GiB for an array with shape (11509, 25088) and data type float32

In [9]:
import matplotlib.image as mpimg
def visualize_original_and_smote(original_dir, features, labels, class_names, num_samples=5):
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 8))

    # Randomly select samples
    random_indices = random.sample(range(len(features)), num_samples)

    for i, idx in enumerate(random_indices):
        # Original image
        class_name = class_names[labels[idx]]
        class_dir = os.path.join(original_dir, class_name)
        original_image = random.choice(os.listdir(class_dir))
        img_path = os.path.join(class_dir, original_image)
        img = mpimg.imread(img_path)
        
        # SMOTE feature map
        feature = features[idx].reshape((7, 7, 512))
        smote_image = np.mean(feature, axis=-1)
        smote_image = (smote_image - np.min(smote_image)) / (np.max(smote_image) - np.min(smote_image))

        # Plot original image
        axes[0, i].imshow(img)
        axes[0, i].axis('off')
        axes[0, i].set_title(f"Original: {class_name}")

        # Plot SMOTE feature
        axes[1, i].imshow(smote_image, cmap='viridis')
        axes[1, i].axis('off')
        axes[1, i].set_title(f"SMOTE: {class_name}")

    plt.tight_layout()
    plt.show()

# Call the function
visualize_original_and_smote(train_dir, X_resampled, y_resampled, class_names)

NameError: name 'X_resampled' is not defined